# 🧠 Employee Attrition Prediction
This notebook analyzes HR data to predict which employees are at risk of leaving the company.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [ ]:
# Load dataset
df = pd.read_csv("WA_Fn-UseC_-HR-Employee-Attrition.csv")
df.head()

In [ ]:
# Explore data
df.info()
df.describe()
df['Attrition'].value_counts()
df.isnull().sum()

In [ ]:
# Visualizations
sns.countplot(data=df, x='Attrition', hue='JobRole')
plt.title('Attrition Distribution by Job Role')
plt.show()

sns.histplot(data=df, x='Age', hue='Attrition', kde=True)
plt.title('Age Distribution by Attrition')
plt.show()

In [ ]:
# Drop unimportant columns
df = df.drop(['EmployeeNumber', 'Over18', 'StandardHours', 'EmployeeCount'], axis=1)

# Convert categorical columns to numeric
df = pd.get_dummies(df, drop_first=True)

In [ ]:
# Define features and label
X = df.drop('Attrition_Yes', axis=1)
y = df['Attrition_Yes']

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Train Random Forest model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

In [ ]:
# Evaluate model
y_pred = model.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))
print('\nClassification Report:\n', classification_report(y_test, y_pred))

sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
# Predict attrition probability for all employees
probabilities = model.predict_proba(X)[:, 1]
df['Attrition_Probability'] = probabilities
df[['Attrition_Probability']].sort_values(by='Attrition_Probability', ascending=False).head()

In [ ]:
# Feature importance
importances = model.feature_importances_
features = X.columns
importance_df = pd.DataFrame({'Feature': features, 'Importance': importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(data=importance_df.head(10), x='Importance', y='Feature')
plt.title('Feature Importance - Factors Influencing Attrition')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()